In [22]:
import numpy as np
import pandas as pd
import math

In [23]:
df = pd.read_excel('bmi_data.xlsx')
df.head()

,Gender,Height,Weight,BMI,Number
0,Male,198,50,12.753801,1
1,Female,198,50,12.753801,2
2,Female,196,50,13.015410,3
3,Female,190,50,13.850416,4
4,Male,190,50,13.850416,5


In [29]:
#Create bins
height_number_of_bins = 10
weight_number_of_bins = 10

height_range_step = (df['Height'].max() - df['Height'].min() + 1)/height_number_of_bins
weight_range_step = (df['Weight'].max() - df['Weight'].min() + 1)/weight_number_of_bins

#Create grid of bins
grid_dim  = (height_number_of_bins, weight_number_of_bins)
bins_grid = np.zeros(grid_dim)

height_column = df['Height']
weight_column = df['Weight']

for i in range(df.shape[0]):
    grid_bin_height_idx = int(( height_column[i] - df['Height'].min() ) // height_range_step)
    grid_bin_weight_idx = int(( weight_column[i] - df['Weight'].min() ) // weight_range_step)
    bins_grid[grid_bin_height_idx][grid_bin_weight_idx] += 1

print(bins_grid)



[[ 1.  3.  7.  8.  1.  2.  3.  6.  8.  3.]
 [ 7.  8.  2.  6.  5.  4.  3.  1.  4.  5.]
 [ 9.  3.  5.  3.  7.  9.  2.  4.  6.  2.]
 [ 4.  4.  3.  1.  6.  4.  6.  5.  5.  9.]
 [ 7.  7.  5.  5.  6.  4.  4.  4. 10.  8.]
 [ 2.  3.  4.  4.  6.  7.  3.  4.  6.  3.]
 [10.  3.  9.  4.  4.  7.  9.  2.  5.  9.]
 [ 5.  4.  8.  8.  8.  3.  7.  8. 11.  7.]
 [ 7.  5.  5.  3.  9.  3.  5.  4.  4.  2.]
 [ 4.  4.  1.  2.  4.  7.  4.  3.  3.  4.]]
